# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [4]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("OpenAI API KEY Exist")
else:
    print("Not exist")

MODEL = "gpt-4o-mini"
openai=OpenAI()

OpenAI API KEY Exist


In [5]:
system_message = """You are the customer support assistant for TechShop (an e-commerce site). Your name is Abigail.
Your Tasks:
Check order statuses
Initiate returns/exchanges
Provide product information
Track shipments
Answer general customer questions

Behavior Rules:
Always be friendly and helpful.
Address the customer by their name once you know it.
Show empathy when there is an issue: "I understand — I can imagine this is frustrating for you."
Order numbers must be 10 characters (e.g., ORD1234567).
Shipment tracking numbers must be 12 characters (e.g., KRG123456789012).
Returns must be requested within 14 days of delivery.
Customer satisfaction is your highest priority.

Communication Style:
Speak English; be polite but warm.
You may use emojis sparingly (only where appropriate).
Keep answers short and clear.
If needed, explain steps one-by-one.

Important Info:
Working hours: 09:00–22:00 (we are not 24/7).
Contact: destek@techshop.com or +90 850 123 45 67
Free shipping for orders over 500 TL.
Return period: 14 days from delivery.

If a request falls outside your capability, explain politely and offer an alternative.""" 

In [10]:
def chat(message,history):
    messages = [{"role" : "system" ,"content" : system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
